<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/CNN_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra # เป็นการ import numpy เพื่อเปิดใช้ฟังก์ชันในการทำงาน
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) # เป็น library ในการจัดการ dataframe
import tensorflow as tf #library สำหรับสร้าง machine learning models แบบ open source จาก Google
import datetime,os # datetime ใช้ในการจัด format ของวัน-เวลาให้อยู่ในรูปแบบที่ต้องการ 
import math # เรียกใช้ module เพื่อคำนวณค่าต่างๆ 
import matplotlib.pyplot as plt # เป็น module พื้นฐานที่ใช้ในการสร้างกราฟของ python
import seaborn as sns # เป็นฟังก์ชันเสริมที่ใช้สร้างกราฟให้มีความสวยงามมากขึ้น
from sklearn.model_selection import train_test_split # เป็น library ในการพัฒนาโปรแกรมโดยใช้การเรียนรู้ของเครื่อง

In [2]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [4]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [5]:
#loading dataframes # เอาที่อยู่ของไฟล์เชื่อมกับที่อยู่ของไฟล์เรา โดยกำหนดชื่่อไฟล์ที่เราต้องการ
train = pd.read_csv(path+"/train.csv")
validation = pd.read_csv(path+"/validation.csv")
test = pd.read_csv(path+"/boneage-test-dataset.csv")

In [6]:
#appending file extension to id column for both training and testing dataframes
# ต่อท้าย data ใน column 'id' ด้วย .png
train['id'] = train['id'].apply(lambda x: str(x)+'.png') # ทำการเปลี่ยน type ใน column 'id' จาก int  เป็น string เพื่อให้สามารถต่อท้ายด้วย .png ได้ # ฟังก์ชันแลมบ์ดาเป็นฟังก์ชันที่ไม่ระบุตัวตนขนาดเล็ก
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png') 
test['id'] = test['id'].apply(lambda x: str(x)+'.png') 

train.head() # แสดงค่าของ train_df บางส่วน

,id,boneage,male
0,1377.png,180,False
1,1378.png,12,False
2,1379.png,94,False
3,1380.png,120,True
4,1381.png,82,False


In [7]:
#finding out the number of male and female children in the dataset
#creating a new column called gender to keep the gender of the child as a string
train['male'] = train['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
validation['male'] = validation['male'].apply(lambda x: 'male' if x else 'female') # ทำการเปลี่ยนค่าข้อมูล gender จาก true -> male, false -> female 
test['male'] = test['male'].apply(lambda x: 'male' if x else 'female')

In [8]:
train = pd.DataFrame(train)
val = pd.DataFrame(validation)
test = pd.DataFrame(test)

In [9]:
train.rename(columns={'male':'gender'},inplace=True)
val.rename(columns={'male':'gender'},inplace=True)
test.rename(columns={'male':'gender'},inplace=True)

In [10]:
mean_bone_age_train = train['boneage'].mean() 
std_bone_age_train = train['boneage'].std()
train['bone_age_z'] = (train['boneage'] - mean_bone_age_train)/(std_bone_age_train)
print(train.head())

         id  boneage  gender  bone_age_z
0  1377.png      180  female    1.278306
1  1378.png       12  female   -2.796338
2  1379.png       94  female   -0.807524
3  1380.png      120    male   -0.176924
4  1381.png       82  female   -1.098570


In [11]:
mean_bone_age_val = val['boneage'].mean() 
std_bone_age_val = val['boneage'].std()
val['bone_age_z'] = (val['boneage'] - mean_bone_age_val)/(std_bone_age_val)
print(val.head())

          id  boneage  gender  bone_age_z
0  10186.png       10    male   -2.864366
1  10758.png       15  female   -2.742400
2  10860.png      180  female    1.282468
3  10877.png      180  female    1.282468
4  11008.png      180  female    1.282468


In [12]:
#import matplotlib.image as mpimg #นำเข้า library ที่ใช้อ่านรูป
#for filename, boneage, gender in train_df[['id','boneage','gender']].sample(10).values: #เป็นคำสั่งวนลูปเพื่อแสดงภาพ และข้อมูลของภาพที่เก็บไว้ใน Train_df (ชื่อ อายุ เพศ)
#    img = mpimg.imread(os.path.join(path,'boneage-training-dataset-5261/')+ filename) # เป็นคำสั่งที่ใช้ในการนำรูปเข้ามา
#    plt.imshow(img) # แสดงรูปภาพ
#    plt.title('Image name:{}  Bone age: {} years  Gender: {}'.format(filename, boneage/12, gender)) # แสดงหัวเรื่องโดยบอกถึง ชื่อ อายุ(ปี) และเพศ ของแต่ละภาพ
#    plt.axis('off')
#    plt.show()

In [13]:
#library required for image preprocessing 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from  keras.applications.xception import preprocess_input

In [14]:
#reducing down the size of the image 
img_size = 256 # ลดขนาดรูปภาพ

In [15]:
train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

In [16]:
#train data generator
train_generator = train_data_generator.flow_from_dataframe(
    dataframe = train,
    directory = (os.path.join(path,'boneage_training_dataset/')),
    x_col= 'id',
    y_col= 'bone_age_z',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode= 'raw',
    #flip_vertical = True,
    color_mode = 'rgb',
    target_size = (img_size, img_size))

Found 10089 validated image filenames.


In [17]:
train_generator

In [18]:
#validation data generator
val_generator = val_data_generator.flow_from_dataframe(
    dataframe = val,
    directory = (os.path.join(path,'boneage_validation_dataset/')),
    x_col = 'id',
    y_col = 'bone_age_z',
    batch_size = 32,
    seed = 42,
    shuffle = True,
    class_mode = 'raw',
    #flip_vertical = True,
    color_mode = 'rgb',
    target_size = (img_size, img_size))

Found 2522 validated image filenames.


In [19]:
#test data generator
test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_data_generator.flow_from_directory(
    directory = (os.path.join(path,'boneage-test-dataset/')),
    target_size = (img_size,img_size),
    shuffle = True,
    class_mode = 'binary',classes=['.'],
    color_mode = 'rgb')

Found 200 images belonging to 1 classes.


#Model

In [20]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [21]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [22]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [23]:
# Define relevant variables for the ML task
START_EPOCH=0
NUM_EPOCHS=60
LR=0.0001
PATIENCE=2
BATCH_SIZE=20
NUM_WORKERS=0
NUM_GPUS=1
GPUS=0

In [24]:
# Device will determine whether to run the training on GPU or CPU.
#device = torch.device('cuda' if torch.cuda.is_available() else 'GPU')


In [25]:
if torch.cuda.is_available():
    device = torch.device('cpu')
else:
    device = torch.device('gpu')

In [26]:
import torchvision.datasets as datasets
from torch.autograd import Variable

In [27]:
# Dataloaders
train_transform = transforms.Compose([transforms.Resize((256, 256)),
                               transforms.RandomAffine(
                                   20, translate=(0.2, 0.2),
                                   scale=(1, 1.2)),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()])

val_transform = transforms.Compose([transforms.Resize((256, 256)),
                               transforms.ToTensor()])

In [34]:
# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)

train_loader

In [29]:
val_loader = torch.utils.data.DataLoader(dataset = val_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)
val_loader

In [30]:
test_loader = torch.utils.data.DataLoader(dataset = test_generator,
                                           batch_size = BATCH_SIZE,
                                           shuffle = True)
test_loader

In [37]:
import torch
import torch.nn as nn

class NN(nn.Module):
  def __init__(self):
      super(NN, self).__init__()
      
      self.layer1 = nn.Sequential(
          torch.nn.Conv2d(in_channels=3, out_channels=32,kernel_size=3),
          torch.nn.ReLU(),
          torch.nn.MaxPool2d(2,2))
      
      self.flattened_tensor = nn.Flatten()

      self.linear_layer = nn.Sequential(
          torch.nn.Linear(128, 64),
          torch.nn.ReLU(),
      )
    
  def forward(self, inp):
    conv_output = self.layer1(inp)
    flattened_output = self.flattened_tensor(conv_output)
    
    transposed_matrix = torch.transpose(flattened_output, 0, 1)
    
    linear_output = self.linear_layer(transposed_matrix)
    return linear_output

model = NN()
print(model)
#output = model()

NN(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flattened_tensor): Flatten(start_dim=1, end_dim=-1)
  (linear_layer): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
  )
)


In [38]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

model = NN().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
loss_func = torch.nn.MSELoss()
criterion = nn.L1Loss()


Using cpu device


#Training

In [39]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(60):
	#Load in the data in batches using the train_loader object
    for i,(images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, NUM_EPOCHS, loss.item()))

            
            


RuntimeError: ignored